In [1]:
import numpy as np
import matplotlib.pyplot as plt
import skimage
import pickle
import random
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu
from skimage.measure import regionprops
from pathlib import Path
from skimage.io import imread
from mtcnn import MTCNN
from math import dist

In [2]:
detector = MTCNN()

In [3]:
path = "../../dataset/train/"
path_val = "../../dataset/valid/"
path_test = "../../dataset/test/"

images_path = list(Path(path).rglob(f"*.jpg"))
images_val = list(Path(path_val).rglob(f"*.jpg"))
images_test = list(Path(path_test).rglob(f"*.jpg"))
label_train = [path.parts[-2] for path in images_path]
label_val = [path.parts[-2] for path in images_val]
label_test = [path.parts[-2] for path in images_test]


In [4]:
def calculate_props(im):
    grey = rgb2gray(im)
    trashold = threshold_otsu(grey)
    binary = grey <= trashold
    binary_labeled = skimage.measure.label(binary)
    props = regionprops(binary_labeled)
    max = 0
    index = 0
    for i in range(len(props)):
        if props[i]['area'] > max:
            max = props[i]['area']
            index = i
    return props[index]['area'], props[index]['perimeter'], props[index]['axis_major_length'], props[index]['axis_minor_length'], props[index]['eccentricity']


In [5]:
def calculate_measures(paths, labels):
    new_labels = []
    new_paths = []
    measures = []
    index = -1
    for i in paths:
        index +=1
        im = imread(i)
        im = (skimage.transform.resize(im, (250,250))*255).astype('uint8')
        face = detector.detect_faces(im)
        if len(face) == 1:
            key = face[0]['keypoints']
            #lef eye
            xleye,yleye =key['left_eye']
            l_eye = (xleye,yleye)
            print(l_eye)
            left_eye = im[yleye-10:yleye+10,xleye-20:xleye+20]
            #right eye
            xrey,yrey =key['right_eye']
            r_eye = (xrey,yrey)
            right_eye = im[yrey-10:yrey+10,xrey-20:xrey+20]
            #nose
            xn,yn =key['nose']
            nose_point = (xn,yn)
            nose = im[yn-25:yn+20,xn-30:xn+30]
            #mouth
            xlmo,ylmo =key['mouth_left']
            xrmo,yrmo =key['mouth_right']
            l_mouth = (xlmo,ylmo)
            r_mouth = (xrmo,yrmo)
            y = min(yrmo,ylmo)
            mouth = im[y-20:y+20,xlmo-10:xrmo+10]
            if len(left_eye[0])>0 and len(right_eye[0])>0 and len(nose[0])>0 and len(mouth[0])>0:
                eye_eye = dist(l_eye, r_eye)
                left_eye_nose = dist(l_eye, nose_point)
                right_eye_nose = dist(r_eye, nose_point)
                nose_left_mouth = dist(nose_point, l_mouth)
                nose_right_mouth = dist(nose_point, r_mouth)
                mouth_mouth = dist(l_mouth, r_mouth)
                eye_color_l1 = np.mean(left_eye[:,:,0])
                eye_color_l2 = np.mean(left_eye[:,:,1])
                eye_color_l3 = np.mean(left_eye[:,:,2])
                eye_color_r1 = np.mean(right_eye[:,:,0])
                eye_color_r2 = np.mean(right_eye[:,:,1])
                eye_color_r3 = np.mean(right_eye[:,:,2])
                left_eye1, left_eye2, left_eye3, left_eye4, left_eye5 = calculate_props(left_eye)
                right_eye1, right_eye2, right_eye3, right_eye4, right_eye5 = calculate_props(right_eye)
                nose1, nose2, nose3, nose4, nose5 = calculate_props(nose)
                mouth1, mouth2, mouth3, mouth4, mouth5 = calculate_props(mouth)
                measures += [[eye_eye, left_eye_nose, right_eye_nose, nose_left_mouth, nose_right_mouth, mouth_mouth,\
                    eye_color_l1, eye_color_l2, eye_color_l3,\
                    eye_color_r1, eye_color_r2, eye_color_r3,\
                    left_eye1, left_eye2, left_eye3, left_eye4, left_eye5,\
                    right_eye1, right_eye2, right_eye3, right_eye4, right_eye5,\
                    nose1, nose2, nose3, nose4, nose5,\
                    mouth1, mouth2, mouth3, mouth4, mouth5]]
                if labels != None:
                    new_paths += [i]
                    new_labels += [labels[index]]
    return measures, new_paths, new_labels

In [9]:
%%capture
measure, images_path, label_train = calculate_measures(images_path, label_train)

In [12]:
with open("./hand_crafted_features/train_feature.pkl","wb") as handle:
    pickle.dump(measure, handle, pickle.HIGHEST_PROTOCOL)
with open("./hand_crafted_features/train_label.pkl", "wb") as handle:
    pickle.dump(label_train, handle, pickle.HIGHEST_PROTOCOL)
with open("./hand_crafted_features/train_image_path.pkl", "wb") as handle:
    pickle.dump(images_path, handle, pickle.HIGHEST_PROTOCOL)

In [6]:
with open("./hand_crafted_features/train_feature.pkl","rb") as handle:
    measure = pickle.load(handle)
with open("./hand_crafted_features/train_label.pkl", "rb") as handle:
    label_train = pickle.load(handle)
with open("./hand_crafted_features/train_image_path.pkl", "rb") as handle:
    images_path = pickle.load(handle)

In [ ]:
%%capture
measure_val, images_val, label_val = calculate_measures(images_val, label_val)

In [ ]:
%%capture
measure_test, images_test, label_test = calculate_measures(images_test, label_test)

In [14]:
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
classifier = svm.NuSVC(nu=0.3,random_state=42)
classifier.fit(measure, label_train)
predicted = classifier.predict(measure_val)
print(classification_report(label_val, predicted))
print(confusion_matrix(label_val, predicted))

NameError: name 'measure_val' is not defined

In [ ]:
predicted = classifier.predict(measure_test)
print(classification_report(label_test, predicted))
print(confusion_matrix(label_test, predicted))

In [7]:
from sklearn.neighbors import KDTree
import os
tree = KDTree(measure)

In [8]:
def imageSubplot(similar):
    import numpy as np
    fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(20, 10))
    for i, path in enumerate(similar):
        plt.subplot(2, 5, i+1)
        plt.axis('off')
        plt.imshow(plt.imread(path))
    plt.show()

In [10]:
for i in range(6,7):
    img = imread(images_test[i])
    feature, _, _ = calculate_measures([images_test[i]], None)
    dist, similar = tree.query(feature, k=10, return_distance=True)
    print(similar)
    #imgs_path = [os.path.join(*images_test[j].split('\\')[-2:]) for j in similar[0]]
    imgs_path = [images_path[j] for j in similar[0]]
    plt.figure()
    plt.imshow(img)
    imageSubplot(imgs_path)

1/1 [==============================] - 0s 11ms/step
(53, 108)


TypeError: 'numpy.ndarray' object is not callable

In [11]:
images_test

[WindowsPath('../../dataset/test/savory/001.jpg'),
 WindowsPath('../../dataset/test/savory/002.jpg'),
 WindowsPath('../../dataset/test/savory/003.jpg'),
 WindowsPath('../../dataset/test/savory/004.jpg'),
 WindowsPath('../../dataset/test/savory/005.jpg'),
 WindowsPath('../../dataset/test/savory/006.jpg'),
 WindowsPath('../../dataset/test/savory/007.jpg'),
 WindowsPath('../../dataset/test/savory/008.jpg'),
 WindowsPath('../../dataset/test/savory/009.jpg'),
 WindowsPath('../../dataset/test/savory/010.jpg'),
 WindowsPath('../../dataset/test/savory/011.jpg'),
 WindowsPath('../../dataset/test/savory/012.jpg'),
 WindowsPath('../../dataset/test/savory/013.jpg'),
 WindowsPath('../../dataset/test/savory/014.jpg'),
 WindowsPath('../../dataset/test/savory/015.jpg'),
 WindowsPath('../../dataset/test/savory/016.jpg'),
 WindowsPath('../../dataset/test/savory/017.jpg'),
 WindowsPath('../../dataset/test/savory/018.jpg'),
 WindowsPath('../../dataset/test/savory/019.jpg'),
 WindowsPath('../../dataset/tes

In [43]:
imgs_path = [images_path[j] for j in similar[0]]


..\..\dataset\train\unsavory\0310.jpg


array([ 5962,  8068,  7826,  8987,  2121,  2643,  1507, 10450,  2925,
        3618], dtype=int64)

In [35]:
for j in similar[0]:
    print(j)
    print(images_path[j])

5962
..\..\dataset\train\unsavory\0310.jpg
8068
..\..\dataset\train\unsavory\2420.jpg
7826
..\..\dataset\train\unsavory\2176.jpg
8987
..\..\dataset\train\unsavory\3342.jpg
2121
..\..\dataset\train\savory\2127.jpg
2643
..\..\dataset\train\savory\2650.jpg
1507
..\..\dataset\train\savory\1510.jpg
10450
..\..\dataset\train\unsavory\4808.jpg
2925
..\..\dataset\train\savory\2932.jpg
3618
..\..\dataset\train\savory\3626.jpg
